In [4]:
path_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/"
path_results_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/"

Fonctions pour nettoyer les datas

In [3]:
import pandas as pd
import numpy as np

In [2]:
from Tool_Functions.cleaning_data import *
from Tool_Functions.join_data import *
from Tool_Functions.test_comportment_reabo import *
from Tool_Functions.visual import *
from Tool_Functions.comportment_reabo import *

In [14]:
def upload_clean(data_path):
    """
    Download clean files in your data_path
    There has to be the dirty files in your data_path
    """
    df = file_to_dataframe(data_path + "Correspondances_Promos.csv",";")
    df_Correspondances_Promos = change_dates_all(df,['DEBVAL', 'FINVAL', 'DEBABOMIN', 'DEBABOMAX'])
    save_to_csv_file(df_Correspondances_Promos,data_path + "df_Correspondances_Promos.csv")

    for i in range(1, 4, 1):
        df = file_to_dataframe(data_path + f"Donnees_Promos_202{i}.csv",",")
        df_Donnees_Promos_202i = clean_dates(df)
        save_to_csv_file(df_Donnees_Promos_202i, data_path + f"df_Donnees_Promos_202{i}.csv")

        df = file_to_dataframe(data_path + f"Donnees_Reabos_202{i}.csv",",")
        df_Donnees_Reabos_202i = clean_dates(df)
        save_to_csv_file(df_Donnees_Reabos_202i,data_path + f"df_Donnees_Reabos_202{i}.csv")

    return

def concat_all_years(data_path):
    """
    Create new csv files that concats the 3 years at once.
    """
    for name in ["df_Donnees_Promos_202", "df_Donnees_Reabos_202"]:
        df1 = file_to_dataframe(data_path + name + "1.csv",",")
        df2 = file_to_dataframe(data_path + name + "2.csv",",")
        df3 = file_to_dataframe(data_path + name + "3.csv",",")

        df = pd.concat([df1, df2, df3], axis=0, ignore_index=True)

        df.to_csv(data_path + name[:-4] + ".csv", index = True)


    return

upload_clean(path_antoine)
concat_all_years(path_antoine)

Partie sur les ODD 

In [11]:
def creation_df_odd(data_path, data_path_results):
    """
    This function creates the new df_odd on your Computer where all the ODD are presented and classified with their type
    """

    df_Correspondances_Promos = file_to_dataframe(data_path + "df_Correspondances_Promos.csv",",") 

    df_odd = df_filter_condition(df_Correspondances_Promos,'TYPE_PROMO','ODD') #we create a DataFrame with only ODD Promotion
    df_odd['TYPE_PROMON'] = create_new_column(df_odd,apply_conditions) #we create new columns on this DataFrame of the ODD type


    for i in [1, 2, 3]:
    #creation df_odd_202i pour chaque année
        df_Donnees_Promos_202i = file_to_dataframe(data_path + f"df_Donnees_Promos_202{i}.csv",",")

        n = df_Donnees_Promos_202i.shape[0] / 10000 #number minimum of used
        df_new_odd = keep_used_odd(df_Donnees_Promos_202i,df_odd,n) #creation of the new tab by keeping only the used promos
        
        save_to_csv_file(df_new_odd,data_path_results + f"odd_202{i}.csv")
    
    #idem sur les trois années
    df_Donnees_Promos = file_to_dataframe(data_path + "df_Donnees_Promos.csv",",")

    n = df_Donnees_Promos.shape[0] / 10000 #number minimum of used
    df_new_odd = keep_used_odd(df_Donnees_Promos,df_odd,n) #creation of the new tab by keeping only the used promos
    
    save_to_csv_file(df_new_odd,data_path_results + f"odd.csv")

    return True


#Intermediary step we change the df_Données_Promos by adding a column 'TYPE_PROMON' for being easier to understand instead of 'CPROMO'

def create_df_Données_Promos_odd(data_path, data_path_results):


    for i in [1, 2, 3]:
        #année i
        df_Donnees_Promos_202i = file_to_dataframe(data_path + f"df_Donnees_Promos_202{i}.csv",",")
        df_odd = file_to_dataframe(data_path + f"odd_202{i}.csv", ",")
        df_Donnees_Promos_202i_odd = join_dataFrames(df_Donnees_Promos_202i,df_odd[['CPROMO','TYPE_PROMON']] ,'CPROMO') #We create a new column 'TYPE_PROMON' on df_Données_Promos_202i
        save_to_csv_file(df_Donnees_Promos_202i_odd,data_path_results + f"df_Donnees_Promos_202{i}_odd.csv") #we save it on your Mac

    return True

def create_df_Données_Promos_odd_all(data_path, data_path_results):
    """
    This function create df_Données_Promos_odd for the dataFrame with all years
    """
    
    df_Donnees_Promos = file_to_dataframe(data_path + "df_Donnees_Promos.csv",",")
    df_odd = file_to_dataframe(data_path + "odd.csv", ",")
    df_Donnees_Promos_odd = join_dataFrames(df_Donnees_Promos,df_odd[['CPROMO','TYPE_PROMON']] ,'CPROMO') #We create a new column 'TYPE_PROMON' on df_Données_Promos_202i
    save_to_csv_file(df_Donnees_Promos_odd,data_path_results + "df_Donnees_Promos_odd.csv") #we save it on your Mac
    
    return True

#Intermediary step we create df_Données_Réabos_odd where there are all the Reabos which corresponds to a reabo

def create_df_Données_Reabos_odd_all(data_path, data_path_results):
    """
    This function create df_Données_Reabos_odd with all years of Reabos which corresponds to a use of Promo
    and then we drop some unused column
    """
    df_Donnees_Promos_odd = file_to_dataframe(data_path +"df_Donnees_Promos_odd.csv" )
    df_Donnees_Reabos = file_to_dataframe(data_path + "df_Donnees_Reabos.csv")
    df_Donnees_Reabos_odd = join_dataFrames(df_Donnees_Promos_odd,df_Donnees_Reabos,['ID_ABONNE','DATE_ACTE_REEL'])

    df_Donnees_Reabos_odd = df_Donnees_Reabos_odd.drop(columns = ["REABO_APRES_ECHEANCE","CPROMO","SECTEUR","PAYS","NUMDIST_PARTENAIRE","NOM_PARTENAIRE","NUMDIST_POINT_DE_VENTE","NOM_POINT_DE_VENTE"])

    end_abo = 'DATE_FIN_ABO_PREC'
    date_reabo = 'DATE_ACTE_REEL'

    df_Donnees_Reabos_odd = time_reabo_columns(df_Donnees_Reabos_odd,end_abo,date_reabo)
    
    save_to_csv_file(df_Donnees_Reabos_odd,data_path_results + "df_Donnees_Reabos_odd.csv")

    return True

creation_df_odd(path_antoine,path_results_antoine)
create_df_Données_Promos_odd(path_antoine, path_results_antoine)
create_df_Données_Promos_odd_all(path_antoine,path_results_antoine)
create_df_Données_Reabos_odd_all(path_antoine,path_results_antoine)

Pourcentages

In [14]:
def repartition_reabo_cond(data_path, data_path_results, action = ['write']):
    """
    This function is used to provide some statistics on the reabo habits.
    action is a list of the actions we need to do.
    """

    df_Donnees_Promos_odd = file_to_dataframe(data_path + "df_Donnees_Promos_odd.csv") #We open the df_Données_Promos_2021_odd where TYPEPROMO <-> CPROMO
    df_Donnees_Reabos = file_to_dataframe(data_path + "df_Donnees_Reabos.csv")
    df_join = join_dataFrames(df_Donnees_Promos_odd,df_Donnees_Reabos,['ID_ABONNE','DATE_ACTE_REEL']) #We join the tables

    df_join = df_mois_annee(df_join,'DATE_ACTE_REEL')

    #We compute some statistcs using count_abo_conditions : this functions count the number of an occurence where the datas are group by conditions 
    df_repartition_promo = count_abo_conditions(df_join,['TYPE_PROMON'],'ID_ABONNE')
    df_type_promo_canaldistrib = count_abo_conditions(df_join,['TYPE_PROMON', 'CANAL_DISTRIB'],'ID_ABONNE')
    df_month_canaldistrib = count_abo_conditions(df_join,['MONTH', 'YEAR', 'CANAL_DISTRIB'],'ID_ABONNE')
    df_repartition_canaldistrib = count_abo_conditions(df_join,['TYPE_PROMON','MONTH', 'YEAR', 'CANAL_DISTRIB'],'ID_ABONNE')
    df_repartition_region = count_abo_conditions(df_join,['TYPE_PROMON','MONTH','YEAR', 'REGION'],'ID_ABONNE')
    df_repartition_secteur = count_abo_conditions(df_join,['TYPE_PROMON','MONTH','YEAR', 'SECTEUR'],'ID_ABONNE')
    df_repartition_enseigne = count_abo_conditions(df_join,['TYPE_PROMON','MONTH','YEAR', 'ENSEIGNE'],'ID_ABONNE')
    df_repartition_moypay = count_abo_conditions(df_join,['TYPE_PROMON','MONTH','YEAR', 'MOYEN_PAIEMENT'],'ID_ABONNE')
    df_repartition_formule = count_abo_conditions(df_join,['TYPE_PROMON','MONTH','YEAR', 'FORMULE_PREC'],'ID_ABONNE')
    
    nouns = ["df_repartition_promo.csv", "type_promo_canaldistrib.csv", "month_canaldistrib.csv", "repartition_canaldistrib.csv", "repartition_region.csv", "repartition_secteur.csv", "repartition_enseigne.csv", "repartition_moypay.csv", "repartition_formule.csv"]
    with open(data_path_results + 'Files_names.txt', "w") as file:
        for element in nouns:
            file.write(f"{element}\n")

    if 'write' in action:
        save_to_csv_file(df_repartition_promo,data_path_results + "df_repartition_promo.csv")
        save_to_csv_file(df_type_promo_canaldistrib,data_path_results + "type_promo_canaldistrib.csv")
        save_to_csv_file(df_month_canaldistrib,data_path_results + "month_canaldistrib.csv")
        save_to_csv_file(df_repartition_canaldistrib,data_path_results + "repartition_canaldistrib.csv")
        save_to_csv_file(df_repartition_region,data_path_results + "repartition_region.csv")
        save_to_csv_file(df_repartition_secteur,data_path_results + "repartition_secteur.csv")
        save_to_csv_file(df_repartition_enseigne,data_path_results + "repartition_enseigne.csv")
        save_to_csv_file(df_repartition_moypay,data_path_results + "repartition_moypay.csv")
        save_to_csv_file(df_repartition_formule,data_path_results + "repartition_formule.csv")

    return True


In [15]:
repartition_reabo_cond(path_antoine, path_results_antoine, action = ['write'])

True

In [26]:
def stats_percentage_one_cond(data_path,data_path_results):
    
    #open_new df_données_Reabos_odd
    df_join = file_to_dataframe(data_path + "df_Donnees_Reabos_odd.csv")

    #We compute some statistcs using count_abo_conditions : this functions count the number of an occurence where the datas are group by conditions 
    df_repartition_promo = percent_abo_conditions(df_join,['TYPE_PROMON'],'ID_ABONNE')
    df_repartition_canaldistrib = percent_abo_conditions(df_join,['CANAL_DISTRIB'],'ID_ABONNE')
    df_repartition_region = percent_abo_conditions(df_join,['REGION'],'ID_ABONNE')
    df_repartition_enseigne = percent_abo_conditions(df_join,['ENSEIGNE'],'ID_ABONNE')
    df_repartition_moypay = percent_abo_conditions(df_join,['MOYEN_PAIEMENT'],'ID_ABONNE')
    df_repartition_formule = percent_abo_conditions(df_join,['FORMULE_PREC'],'ID_ABONNE')

    save_to_csv_file(df_repartition_promo,data_path_results + "repartition_promo_%.csv")
    save_to_csv_file(df_repartition_canaldistrib,data_path_results + "repartition_canaldistrib_%.csv")
    save_to_csv_file(df_repartition_region,data_path_results + "repartition_region_%.csv")
    save_to_csv_file(df_repartition_enseigne,data_path_results + "repartition_enseigne_%.csv")
    save_to_csv_file(df_repartition_moypay,data_path_results + "repartition_moypay_%.csv")
    save_to_csv_file(df_repartition_formule,data_path_results + "repartition_formule_%.csv")
    
    return True

In [17]:
stats_percentage_one_cond(path_antoine,path_results_antoine)

True

In [19]:
def stats_percentage_multiple_conds(data_path,data_path_results):
    
    #open_new df_données_Reabos_odd
    df_join = file_to_dataframe(data_path + "df_Donnees_Reabos_odd.csv")

    
    #We compute some statistcs using count_abo_conditions : this functions count the number of an occurence where the datas are group by conditions 
    df_repartition_canaldistrib = percent_abo_conditions_group(df_join,['TYPE_PROMON','CANAL_DISTRIB'],'ID_ABONNE')
    df_repartition_region = percent_abo_conditions_group(df_join,['TYPE_PROMON','REGION'],'ID_ABONNE')
    df_repartition_enseigne = percent_abo_conditions_group(df_join,['TYPE_PROMON','ENSEIGNE'],'ID_ABONNE')
    df_repartition_moypay = percent_abo_conditions_group(df_join,['TYPE_PROMON','MOYEN_PAIEMENT'],'ID_ABONNE')
    df_repartition_formule = percent_abo_conditions_group(df_join,['TYPE_PROMON','FORMULE_PREC'],'ID_ABONNE')

    save_to_csv_file(df_repartition_canaldistrib,data_path_results + "promo_" + "repartition_canaldistrib.csv")
    save_to_csv_file(df_repartition_region,data_path_results +  "promo_" +"repartition_region.csv")
    save_to_csv_file(df_repartition_enseigne,data_path_results + "promo_" + "repartition_enseigne.csv")
    save_to_csv_file(df_repartition_moypay,data_path_results  + "promo_"+ "repartition_moypay.csv")
    save_to_csv_file(df_repartition_formule,data_path_results + "promo_" + "repartition_formule_prec.csv")
    
    return True

In [20]:
stats_percentage_multiple_conds(path_antoine,path_results_antoine)

True

In [32]:
def one_cond_compare_months_years(data_path, data_path_results):

    "Compare les utilisations d'un type au cours des mois et des années, indépendamment des promos"
    
    df_join = file_to_dataframe(data_path + "df_Donnees_Reabos_odd.csv")
    df_join = df_mois_annee(df_join,'DATE_ACTE_REEL')

    df_month_canaldistrib = count_abo_conditions(df_join,['MONTH', 'YEAR', 'CANAL_DISTRIB'],'ID_ABONNE')
    df_month_promo = count_abo_conditions(df_join,['MONTH', 'YEAR', 'TYPE_PROMON'],'ID_ABONNE')
    df_month_region = count_abo_conditions(df_join,['MONTH', 'YEAR', 'REGION'],'ID_ABONNE')
    df_month_enseigne = count_abo_conditions(df_join,['MONTH', 'YEAR', 'ENSEIGNE'],'ID_ABONNE')
    df_month_moypay = count_abo_conditions(df_join,['MONTH', 'YEAR', 'MOYEN_PAIEMENT'],'ID_ABONNE')
    df_month_formule = count_abo_conditions(df_join,['MONTH', 'YEAR', 'FORMULE_PREC'],'ID_ABONNE')

    save_to_csv_file(df_month_canaldistrib,data_path_results + "month_canaldistrib.csv")
    save_to_csv_file(df_month_promo,data_path_results + "month_promo.csv")
    save_to_csv_file(df_month_region,data_path_results + "month_region.csv")
    save_to_csv_file(df_month_enseigne,data_path_results + "month_enseigne.csv")
    save_to_csv_file(df_month_moypay,data_path_results + "month_moypay.csv")
    save_to_csv_file(df_month_formule,data_path_results + "month_formule.csv") 

    return True

In [33]:
one_cond_compare_months_years(path_antoine, path_results_antoine)

True

12 mois glissants